## Setup

In [ ]:
# Standard modules
import pandas as pd
import numpy as np
import time
from datetime import datetime
import thefuzz
from thefuzz import process

import geopandas as gpd
import geopy as gp
import shapely as shp
from shapely import wkt
from shapely.geometry import LineString
import pyproj # Used to convert coordinate projections

from sqlalchemy import create_engine, text
from os import environ
import mysql.connector

import folium
from folium.plugins import MarkerCluster

In [ ]:
# Create engine
engine = environ.get("sqlalchemy_engine")

engine = create_engine(engine)

## Time Static Map
### Data for Static Map

In [30]:
# SQL query to get data
query_static = '''WITH temp AS (
SELECT		DISTINCT curb_id
FROM		nyc_parking.block_sign bs
LEFT JOIN	nyc_parking.curb c
ON			bs.physicalid = c.physicalid AND
			bs.sign_side = c.curb_side
LEFT JOIN	nyc_parking.sign s
ON			bs.sign_id = s.sign_id
LEFT JOIN	nyc_parking.sign_description sd
ON			s.sign_code = sd.sign_code
LEfT JOIN	nyc_parking.restrict r
ON			sd.restrict_id = r.restrict_id 
GROUP BY	curb_id
HAVING		COUNT(DISTINCT r.restrict_id) = 1 )

SELECT		DISTINCT ST_ASTEXT(c.geometry) AS geometry,
			r.*
FROM		nyc_parking.block_sign bs
LEFT JOIN	nyc_parking.curb c
ON			bs.physicalid = c.physicalid AND
			bs.sign_side = c.curb_side
LEFT JOIN	nyc_parking.sign s
ON			bs.sign_id = s.sign_id
LEFT JOIN	nyc_parking.sign_description sd
ON			s.sign_code = sd.sign_code
LEfT JOIN	nyc_parking.restrict r
ON			sd.restrict_id = r.restrict_id 
WHERE		curb_id IN (SELECT curb_id FROM temp) ;'''

# Execute the query and load data into a DataFrame
df_static = pd.read_sql(query_static, engine)

# Set the geometry column
df_static['geometry'] = df_static['geometry'].apply(wkt.loads)

# Project the CRS to 4326 (latitude and longitude)
gdf_static = gpd.GeoDataFrame(df_static, geometry='geometry', crs='EPSG:2263').to_crs(4326)

# Calculate the number of hours after midnight the restriction begins and ends
gdf_static['start_time_num'] = gdf_static['start_time'].dt.total_seconds()/3600
gdf_static['end_time_num'] = gdf_static['end_time'].dt.total_seconds()/3600

In [ ]:
# Separate GeoDataFrames by restriction color relative to 11:30 on Monday
gdf_red     = gdf_static.loc[(gdf_static.start_time_num <= 11.5) & (gdf_static.end_time_num >= 11.5) & (gdf_static.monday == 1), 'geometry']
gdf_yellow  = gdf_static.loc[(gdf_static.start_time_num - .5 <= 11.5) & (gdf_static.start_time_num > 11.5) & (gdf_static.monday == 1), 'geometry']
gdf_green   = gdf_static.loc[(((gdf_static.start_time_num - .5 > 11.5) | (gdf_static.end_time_num < 11.5)) & (gdf_static.monday == 1)) | (gdf_static.monday == 0), 'geometry']

### Static Map

In [ ]:
# Create map
m_static = folium.Map(location=[40.67709461762126, -73.9386906766971], zoom_start=16)

# Add GeoDataFrames to the map
folium.GeoJson(gdf_red, name="Layer 1", style_function=lambda x: {"color": "red"}).add_to(m_static)
folium.GeoJson(gdf_yellow, name="Layer 2", style_function=lambda x: {"color": "yellow"}).add_to(m_static)
folium.GeoJson(gdf_green, name="Layer 3", style_function=lambda x: {"color": "green"}).add_to(m_static)

# Add layer control
folium.LayerControl().add_to(m_static)


In [24]:
# Save the map as html
m_static.save('static.html')

## Time Dynamic Map
### Data for Dynamic Map

In [ ]:

# Write your SQL query
query = "SELECT		DISTINCT ST_ASTEXT(c.geometry) AS geometry, r.* FROM nyc_parking.block_sign bs LEFT JOIN nyc_parking.curb c ON bs.physicalid = c.physicalid AND bs.sign_side = c.curb_side LEFT JOIN nyc_parking.sign s ON bs.sign_id = s.sign_id LEFT JOIN	nyc_parking.sign_description sd ON s.sign_code = sd.sign_code LEfT JOIN	nyc_parking.restrict r ON sd.restrict_id = r.restrict_id WHERE bs.physicalid = 38533 ;"

# Execute the query and load data into a DataFrame
df = pd.read_sql(query, engine)

df['geometry'] = df['geometry'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:2263').to_crs(4326)

### Dynamic Map

In [28]:
# Create map
m_dynamic = folium.Map(location=[40.67709461762126, -73.9386906766971], zoom_start=24)

# Create timestamped geojson
lines = [
    # North curb
    {
        "coordinates": [
            [-73.9386906766971, 40.67709461762126], 
            [-73.93591915719765, 40.67693913714084],
        ],
        "dates": ["2024-12-09T10:00:00", "2024-12-09T10:00:00"],
        "color": "green",
    },
    {
        "coordinates": [
            [-73.9386906766971, 40.67709461762126], 
            [-73.93591915719765, 40.67693913714084],
        ],
        "dates": ["2024-12-09T10:00:00", "2024-12-09T11:00:00"],
        "color": "yellow",
    },
    {
        "coordinates": [
            [-73.9386906766971, 40.67709461762126], 
            [-73.93591915719765, 40.67693913714084],
        ],
        "dates": ["2024-12-09T11:00:00", "2024-12-09T11:30:00"],
        "color": "red",
    },
    {
        "coordinates": [
            [-73.9386906766971, 40.67709461762126], 
            [-73.93591915719765, 40.67693913714084],
        ],
        "dates": ["2024-12-09T11:30:00", "2024-12-09T13:00:00"],
        "color": "green",
    },

    # South curb
        {
        "coordinates": [
            [-73.9359276370314, 40.67685154215516], 
            [-73.93869915201554, 40.67700702238247],
        ],
        "dates": ["2024-12-09T10:00:00", "2024-12-09T10:00:00"],
        "color": "green",
    },
    {
        "coordinates": [
            [-73.9359276370314, 40.67685154215516], 
            [-73.93869915201554, 40.67700702238247],
        ],
        "dates": ["2024-12-09T10:00:00", "2024-12-10T11:00:00"],
        "color": "yellow",
    },
    {
        "coordinates": [
            [-73.9359276370314, 40.67685154215516], 
            [-73.93869915201554, 40.67700702238247],
        ],
        "dates": ["2024-12-10T11:00:00", "2024-12-10T11:30:00"],
        "color": "red",
    },
    {
        "coordinates": [
            [-73.9359276370314, 40.67685154215516], 
            [-73.93869915201554, 40.67700702238247],
        ],
        "dates": ["2024-12-10T11:30:00", "2024-12-10T13:00:00"],
        "color": "green",
    },
]

# Specify design elements of the map
features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "times": line["dates"],
            "style": {
                "color": line["color"],
                "weight": line["weight"] if "weight" in line else 5,
            },
        },
    }
    for line in lines
]

# Add design elements to the map
folium.plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT30M",
    add_last_point=True,
).add_to(m_dynamic)

In [ ]:
# Save the map as html
m_dynamic.save('dynamic.html')